In [1]:
from IPython.utils import io
import pandas as pd
import os
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import seaborn as sns
import time
import joblib
from os.path import exists
import shutil
import sys
import time

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
    
#From my EEG package 
import run_expts
import constants


#Let me see as many results as I want to see
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 20)

In [2]:
load_path = '/export/sphere/ebirah/ko20929/RBD_using_custom_package/Data/Catch_22_features/'

In [3]:
X = pd.read_hdf(load_path + 'REM_c_22_feautures.h5', key='df', mode='r')
y = pd.read_hdf(load_path + 'REM_y.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + 'REM_groups.h5', key='df', mode='r')

#1.Generate expt specific X,y,groups
X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 1 )

# # #One channel per region is worse than three channels 
# X_cols = X_expt.columns
# one_chan_per_reg = [channels[1] for channels in reg_chs]
# selected_cols = [col for col in X_cols if any(col.startswith(prefix) for prefix in one_chan_per_reg)]
# X_expt = X_expt[selected_cols]

X_expt = X_expt[ [col for col in X_expt.columns if 'Fpz' == col[:3] or 'FCz'== col[:3] or 'POz'== col[:3] ] ]

# X_expt = X_expt[ [col for col in X_expt.columns if chan == col[:len(chan)] for chan in channels ]]

In [4]:
X_expt

,Fpz_DN_HistogramMode_5,Fpz_DN_HistogramMode_10,Fpz_SB_BinaryStats_diff_longstretch0,Fpz_DN_OutlierInclude_p_001_mdrmd,Fpz_DN_OutlierInclude_n_001_mdrmd,Fpz_CO_f1ecac,Fpz_CO_FirstMin_ac,Fpz_SP_Summaries_welch_rect_area_5_1,Fpz_SP_Summaries_welch_rect_centroid,Fpz_FC_LocalSimple_mean3_stderr,...,POz_SB_BinaryStats_mean_longstretch1,POz_SB_MotifThree_quantile_hh,POz_FC_LocalSimple_mean1_tauresrat,POz_CO_Embed2_Dist_tau_d_expfit_meandiff,POz_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,POz_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,POz_SB_TransitionMatrix_3ac_sumdiagcov,POz_PD_PeriodicityWang_th0_01,POz_StandardDeviation,POz_Mean
0,-2.841434,0.313940,272.0,0.032390,0.034711,12.0,20.0,21.322590,0.086562,2.272537,...,28.0,1.560258,0.024390,0.090393,0.42,0.50,0.000073,33.0,5.880081,-0.001143
1,1.978857,-0.908866,259.0,-0.002346,-0.001628,12.0,20.0,18.462780,0.080935,2.106292,...,31.0,1.567569,0.018750,0.081298,0.48,0.54,0.000065,31.0,5.509858,-0.003174
2,0.331202,-3.973876,392.0,0.003075,0.002171,38.0,331.0,35.347343,0.031809,1.572912,...,54.0,1.394977,0.060606,0.036330,0.36,0.42,0.000061,30.0,7.301861,0.008359
3,-3.169513,2.199171,530.0,-0.269873,-0.265292,39.0,14.0,35.556252,0.028618,2.020711,...,34.0,1.485292,0.020548,0.043262,0.40,0.48,0.000064,20.0,7.509506,-0.006630
4,-3.186679,1.752790,659.0,-0.218303,-0.216764,56.0,19.0,54.931560,0.023837,1.919861,...,39.0,1.441940,0.058333,0.056963,0.44,0.50,0.000153,25.0,9.112752,0.001066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,-3.664769,0.028143,518.0,-0.034359,-0.025664,29.0,18.0,17.511115,0.052000,1.846215,...,33.0,1.515354,0.020408,0.038718,0.46,0.50,0.000043,42.0,5.208742,0.003892
58,8.107625,2.902621,790.0,-0.162580,-0.158298,79.0,34.0,32.443027,0.015118,1.927585,...,35.0,1.480567,0.013652,0.021838,0.44,0.50,0.000025,36.0,6.273039,-0.001996
59,-4.647871,0.771298,529.0,-0.169006,-0.159588,56.0,340.0,42.874368,0.017617,1.874197,...,42.0,1.404551,0.027972,0.046375,0.44,0.46,0.000137,1376.0,7.836346,0.016600
60,1.504327,-3.097850,384.0,-0.068824,-0.131048,46.0,23.0,48.162232,0.026317,2.004463,...,39.0,1.446457,0.036697,0.105006,0.46,0.52,0.000123,32.0,8.307048,-0.004598


In [10]:
reg_chs=['Fp1', 'Fpz', 'Fp2'], ['F1', 'Fz', 'F2'], ['AF7', 'F7', 'F5', 'F3'], ['AF8', 'F8', 'F6', 'F4'], \
            ['FCz', 'FC1', 'FC2', 'Cz', 'C1', 'C2', 'CP1', 'CP2'], ['FC5', 'FC3', 'C5', 'C3', 'CP5', 'CP3'], \
            ['FC6', 'FC4', 'C6', 'C4', 'CP6', 'CP4'], ['TP7', 'T7', 'FT7', 'FT9'], ['TP8', 'T8', 'FT8', 'FT10'], \
            ['P1', 'P2', 'Pz'], ['P3', 'P5', 'P7', 'PO7'], ['P4', 'P6', 'P8', 'PO8'], ['POz', 'O1', 'O2', 'Oz']

In [11]:
one_chan_per_reg = [channels[1] for channels in reg_chs]
len(one_chan_per_reg)

13

In [13]:
acc_scores = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_expt , y_expt, test_size=0.2)
    clf = RandomForestClassifier()

    t1 = time.time()

    clf.fit(X_train, y_train)
    y_preds = clf.predict(X_test)
    
    acc = accuracy_score(y_preds , y_test )
    

    print(acc )
    
    acc_scores.append(acc)

    t2 = time.time()

mean_acc = np.mean(np.array(acc_scores))
print('mean accuracy is:' + str(mean_acc) )
#     print(t2-t1)

0.75
0.625
0.875
0.625
1.0
0.625
0.875
0.875
0.5
0.5
mean accuracy is:0.725


In [105]:
# Best mean is around 70 with features from three different channels